### Imports

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz
import difflib 
from fuzzywuzzy import process
import time

### Importing inputs
- 3 Mout. Volumes 
- wcvp 
- adding a column with author lists to each dataframe

##### updating author functions

In [13]:
#TODO: move this to index code
def fix_author(authors_field):
    """this function removes the extra space that currently exists
       in the 'author' filed of the index csv files
       INPUT: any value
       OUTPUT: the function returns non string values as is
               and removes last character for string values 
    """
    if not isinstance(authors_field, str) or authors_field[-1] != ' ':
        return authors_field
    return authors_field[:-1]

In [14]:
#TODO: set the list of connectives as an external thing
def split_author(author_str):
    """function that splits the author names ignoring given connectives used
       INPUT: a string (of authors)
       OUTPUT: a list of containing each author from the input string
    """
    if (not isinstance(author_str, str)) or author_str == "np.NaN":
        return author_str
    paran_list = []
    reg_list = []

    in_pran = ""
    out_pran = ""
    #the paranthesis part (assuming we have one paranthesis only)
    search_full_paran = re.search(r"\(.*\)", author_str)
    search_close_paran = re.search(r".*\)", author_str)
    if search_full_paran:
        in_pran = search_full_paran.group(0)[1:-1] 
    elif search_close_paran:
        in_pran = search_close_paran.group(0)[:-1]
    
    split_paran = re.split(r".*\)", author_str, maxsplit=1)
    out_pran = split_paran[int(len(split_paran) == 2)]
    
    #remove anything after ex. 
    in_pran = re.sub(r"\sex[\s|.].*","",in_pran)
    out_pran = re.sub(r"\sex[\s|.].*","",out_pran)


    #ignoring et, in, non, &, er, nec, with space or . after (TODO: check end/ beggining of string)
    ignore_pattern = r"\set[\s|.]|\sin[\s|.]|\snon[\s|.]|\s&[\s|.]|\ser[\s|.]|\snec[\s|.]|\smult[\s|.]"
    paran_list = re.split(ignore_pattern, in_pran) 
    reg_list = re.split(ignore_pattern, out_pran) 
    
    #TODO: improve this part
    return [aut for aut in paran_list + reg_list if aut != ""] 

In [15]:
# a function for flagging authors with auct. / agg. etc
# so that we can just use the wcvp version later
# note, vol3 only has 2 occurances of auct. 
# the rest don't show up because we are only looking at non-italicized part of index

##### importing dataframes 

In [16]:
#mout. books: 
vol1_path = '../output/local/index_output/vol1_nonitalics.csv'
vol2_path = '../output/local/index_output/vol2_nonitalics.csv'
vol3_path = '../output/local/index_output/vol3_nonitalics.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#changing name of columns of mout. indecies 
vol1_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/input copy/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

In [17]:
wcvp_df.columns

Index(['kew_id', 'family', 'genus', 'species', 'infraspecies', 'taxon_name',
       'authors', 'rank', 'taxonomic_status', 'accepted_kew_id',
       'accepted_name', 'accepted_authors', 'parent_kew_id', 'parent_name',
       'parent_authors', 'reviewed', 'publication', 'original_name_id'],
      dtype='object')

In [18]:
vol1_df.columns

Index(['Unnamed: 0', 'mouterde_genus', 'mouterde_epithet', 'mouterde_infra',
       'mouterde_author', 'taxon_rank', 'taxon_rank_detailed'],
      dtype='object')

In [19]:
['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author', 'taxon rank'] + ['kew_id', 'family', 'genus', 'species', 'infraspecies', 'taxon_name',
       'authors', 'rank', 'taxonomic_status', 'accepted_kew_id',
       'accepted_name', 'accepted_authors', 'parent_kew_id', 'parent_name',
       'parent_authors', 'reviewed', 'publication', 'original_name_id']

['mouterde_genus',
 'mouterde_epithet',
 'mouterde_infra',
 'mouterde_author',
 'taxon rank',
 'kew_id',
 'family',
 'genus',
 'species',
 'infraspecies',
 'taxon_name',
 'authors',
 'rank',
 'taxonomic_status',
 'accepted_kew_id',
 'accepted_name',
 'accepted_authors',
 'parent_kew_id',
 'parent_name',
 'parent_authors',
 'reviewed',
 'publication',
 'original_name_id']

In [20]:
wcvp_df.shape[0]

1232931

In [21]:
wcvp_df[wcvp_df['rank'] == 'GENUS']['parent_name'].unique()

array([nan], dtype=object)

In [22]:
wcvp_df['rank'].unique()

array(['GENUS', 'SPECIES', 'SUBSPECIES', 'VARIETY', 'Form', 'Subvariety',
       'Subform', 'InfraspecificName', nan], dtype=object)

##### applying the author functions

In [23]:
#fix spacing issue in Mout. books
# vol1_df['mouterde_author'] = vol1_df['mouterde_author'].apply(fix_author)
# vol2_df['mouterde_author'] = vol2_df['mouterde_author'].apply(fix_author)
# vol3_df['mouterde_author'] = vol3_df['mouterde_author'].apply(fix_author)

#getting author lists
wcvp_df['author_list'] = wcvp_df['authors'].apply(split_author)
vol1_df['mouterde_author_list'] = vol1_df['mouterde_author'].apply(split_author)
vol2_df['mouterde_author_list'] = vol2_df['mouterde_author'].apply(split_author)
vol3_df['mouterde_author_list'] = vol3_df['mouterde_author'].apply(split_author)

### Finding closest Match for the genera

In [24]:
#uses default matching criteria of difflib
def difflib_closest_matches(input_str, matching_lst):
    matched_list = difflib.get_close_matches(input_str, matching_lst)
    if len(matched_list) > 0:
        return matched_list[0]
    else:
        print(input_str)
        return np.NaN

In [28]:
from unidecode import unidecode

vol1_df['mouterde_genus'] = vol1_df['mouterde_genus'].apply(lambda x : unidecode(x) if isinstance(x, str) else np.NaN)
vol2_df['mouterde_genus'] = vol2_df['mouterde_genus'].apply(lambda x : unidecode(x) if isinstance(x, str) else np.NaN)
vol3_df['mouterde_genus'] = vol3_df['mouterde_genus'].apply(lambda x : unidecode(x) if isinstance(x, str) else np.NaN)

In [29]:
wcvp_unique_genera = wcvp_df['genus'].apply(lambda x : unidecode(x) if isinstance(x, str) else np.NaN).unique()
vol1_df.replace(np.NaN, "",inplace = True)
vol2_df.replace(np.NaN, "",inplace = True)
vol3_df.replace(np.NaN, "",inplace = True)

startTime = time.time()
#prints the ones with no closest match
print("---- vol1 genus match ----")
vol1_df['mout_genus_close_match'] = vol1_df['mouterde_genus'].apply(lambda x: difflib_closest_matches(x, wcvp_unique_genera))
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

startTime = time.time()
print("---- vol2 genus match ----")
vol2_df['mout_genus_close_match'] = vol2_df['mouterde_genus'].apply(lambda x: difflib_closest_matches(x, wcvp_unique_genera))
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

startTime = time.time()
print("---- vol3 genus match ----")
vol3_df['mout_genus_close_match'] = vol3_df['mouterde_genus'].apply(lambda x: difflib_closest_matches(x, wcvp_unique_genera))
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

---- vol1 genus match ----

Seconds to Run:101.25555801391602
-------- complete --------

---- vol2 genus match ----
Seconds to Run:109.75097489356995
-------- complete --------

---- vol3 genus match ----
Seconds to Run:130.70312905311584
-------- complete --------



In [232]:
# def get_best_match(x):
#     if isinstance(x, list):
#         if len(x) >0:
#             return x[0]
#         else:
#             return x
#     else:
#         return x

In [233]:
# vol1_df['mout_genus_close_match']

0                  NaN
1       Acanthophyllum
2       Acanthophyllum
3          Achyranthes
4          Achyranthes
             ...      
1378            Vulpia
1379      Zannichellia
1380      Zannichellia
1381           Zostera
1382           Zostera
Name: mout_genus_close_match, Length: 1383, dtype: object

In [30]:
print(vol1_df[vol1_df['mout_genus_close_match'] != vol1_df['mouterde_genus'].str.strip()][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol1_df[vol1_df['mout_genus_close_match'] != vol1_df['mouterde_genus'].str.strip()][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().head(50)

12


,mouterde_genus,mout_genus_close_match
0,,NaN
32,AgropyrQn,Agropyron
215,Asphodelusi,Asphodelus
321,Buffonia,Bufonia
373,Gerastium,Cerastium
384,Geterach,Ceterach
546,Eremopyron,Eremophyton
866,Naias,Nazia
966,Paspatiun,Paspalanthium
968,Penmsetum,Pennisetum


In [31]:
difflib_closest_matches("Paspalum", wcvp_unique_genera)

'Paspalum'

In [32]:
print(vol2_df[vol2_df['mout_genus_close_match'] != vol2_df['mouterde_genus'].str.strip()][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol2_df[vol2_df['mout_genus_close_match'] != vol2_df['mouterde_genus'].str.strip()][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates()

18


,mouterde_genus,mout_genus_close_match
143,Anisociadium,Anisosciadium
313,Aubrietia,Aubrieta
357,Buffonia,Bufonia
436,*Ceratophyllum,Ceratophyllum
451,Ghesneya,Chesneya
460,Chrysocamela,Chrysochamela
466,Gircaea,Dircaea
522,Goronilla,Coronilla
556,Crystopteris,Cystopteris
621,Elaeagtlus,Elaeagnus


In [33]:
print(vol3_df[vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus'].str.strip()][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol3_df[vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus'].str.strip()][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates()

9


,mouterde_genus,mout_genus_close_match
158,BallotaL.,Ballota
210,Calhpeltis,Callipeltis
437,CordiaL.,Cordia
573,Engeron,Erigeron
718,Hypochoeris,Hypochaeris
912,Moriria,Morierina
1219,Satureia,Saturnia
1345,SolidagoL.,Solidago
1477,Trachornitum,Trachomitum


for volume3, use 'genus_matched' (includes both accepted matches and genera in vol3_accepted_genus_changes) to see the closest match of which genera should be used

In [242]:
vol1_accepted_genus_changes = {
    "AgropyrQn"	: "Agropyron",
    "Asphodelusi" :	"Asphodelus",
    "Buffonia"	: "Bufonia",
    "Gerastium" : "Cerastium",
    "Geterach" : "Ceterach",
    "Çutandia" : "Cutandia",
    "Danaë" : "Danae",
    "Eremopyron" : "Eremopyrum",
    "Naias" : "Naias",
    "Paspatiun" : "Paspalum",
    "Penmsetum" : "Pennisetum",
    "Silène" : "Silene",
    "Tseniatherum" : "Taeniatherum",
    "Thelygonum" : "Theligonum"
}
vol2_accepted_genus_changes = {}

vol3_accepted_genus_changes = {
    "Anthémis" : "Anthemis",
    "BallotaL." : "Ballota",
    "CordiaL." : "Cordia",
    "Crépis" : "Crepis",
    "SolidagoL." : "Solidago"
}

In [243]:
vol1_df['genus_matched'] = (vol1_df['mout_genus_close_match'] == vol1_df['mouterde_genus']) | (vol1_df['mouterde_genus'].isin(vol1_accepted_genus_changes))
vol2_df['genus_matched'] = (vol2_df['mout_genus_close_match'] == vol2_df['mouterde_genus']) | (vol2_df['mouterde_genus'].isin(vol2_accepted_genus_changes))
vol3_df['genus_matched'] = (vol3_df['mout_genus_close_match'] == vol3_df['mouterde_genus']) | (vol3_df['mouterde_genus'].isin(vol3_accepted_genus_changes))

In [244]:
vol1_df

,Unnamed: 0,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon_rank,taxon_rank_detailed,mouterde_author_list,mout_genus_close_match,genus_matched
0,1,,cilicica,,Ant. et Ky,epithet,epithet,"[Ant., Ky]",NaN,False
1,5,Acanthophyllum,,,,genus,genus,,Acanthophyllum,True
2,6,Acanthophyllum,kurdicum,,Boiss. et Hausskn.,epithet,epithet,"[Boiss., Hausskn.]",Acanthophyllum,True
3,23,Achyranthes,,,,genus,genus,,Achyranthes,True
4,24,Achyranthes,aspera,,L.,epithet,epithet,[L.],Achyranthes,True
...,...,...,...,...,...,...,...,...,...,...
1378,5473,Vulpia,uniglumis,,(Sol.) Dumort.,epithet,epithet,"[Sol., Dumort.]",Vulpia,True
1379,5481,Zannichellia,,,,genus,genus,,Zannichellia,True
1380,5482,Zannichellia,palustris,,L.,epithet,epithet,[L.],Zannichellia,True
1381,5495,Zostera,,,,genus,genus,,Zostera,True


### For each species, find the closes match within it's closest match genera

In [245]:
#uses default matching criteria of difflib
def difflib_closest_epithet_match(row):
    epithet = row['mouterde_epithet']
    if row['genus_matched'] and isinstance(epithet, str):
        genus = row['mout_genus_close_match']
        matching_list = wcvp_df[wcvp_df['genus'] == genus]['species'].unique()
        cleanedList = [x for x in matching_list if str(x) != 'nan']
        match_result = difflib.get_close_matches(epithet, cleanedList)  
        
        if match_result:
            return match_result[0]
        else:
            return np.NaN
    else: 
        return np.NaN

In [35]:
startTime = time.time()
#prints the ones with no closest match
print("---- vol1 epithet match ----")
vol1_df['mout_epithet_close_match'] = vol1_df.apply(difflib_closest_epithet_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("--------- complete ---------\n")

startTime = time.time()
print("---- vol2 epithet match ----")
vol2_df['mout_epithet_close_match'] = vol2_df.apply(difflib_closest_epithet_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("--------- complete ---------\n")

startTime = time.time()
print("---- vol3 epithet match ----")
vol3_df['mout_epithet_close_match'] = vol3_df.apply(difflib_closest_epithet_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("--------- complete ---------\n")

---- vol1 epithet match ----


NameError: name 'difflib_closest_epithet_match' is not defined

##### Places where non NaN matches of the epithet do not overlap and the number of such occurances

In [34]:
print(vol1_df[(vol1_df['mout_epithet_close_match'] != vol1_df['mouterde_epithet'].str.strip()) & ~(vol1_df['mout_epithet_close_match'].isna())][['mout_genus_close_match','mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates().shape[0])
vol1_df[(vol1_df['mout_epithet_close_match'] != vol1_df['mouterde_epithet'].str.strip()) & ~(vol1_df['mout_epithet_close_match'].isna())][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates().head(50)

KeyError: 'mout_epithet_close_match'

In [30]:
print(vol2_df[(vol2_df['mout_epithet_close_match'] != vol2_df['mouterde_epithet'].str.strip()) & ~(vol2_df['mout_epithet_close_match'].isna())][['mout_genus_close_match','mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates().shape[0])
vol2_df[(vol2_df['mout_epithet_close_match'] != vol2_df['mouterde_epithet'].str.strip()) & ~(vol2_df['mout_epithet_close_match'].isna())][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

102


,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
6,Acer,tauricolum,tauricum
58,Alchemilla,noribunda,floribunda
62,Alhagi,pseudalhâgi,pseudalhagi
74,Althaea,bertrami,bertramii
98,Alyssum,stapffii,stapfii
...,...,...,...
1390,Polypogon,semiyerticillatus,semiverticillatus
1426,Psammogeton,setifolium,setifolius
1433,Pteris,vïttata,vittata
1452,Ranunculus,macrorrhynchup,macrorrhynchus


In [32]:
print(vol3_df[(vol3_df['mout_epithet_close_match'] != vol3_df['mouterde_epithet']) & ~(vol3_df['mout_epithet_close_match'].isna())][['mout_genus_close_match','mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates().shape[0])
vol3_df[(vol3_df['mout_epithet_close_match'] != vol3_df['mouterde_epithet']) & ~(vol3_df['mout_epithet_close_match'].isna())][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

45


,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
80,Anthemis,eretica,cretica
91,Anthemis,palestina,palaestina
127,Artemisia,verlotorum,verlotiorum
157,Atropa,belladonna,bella-donna
322,Centaurea,ptosimoppapa,ptosimopappa
349,Centranthus,longinorus,longiflorus
401,Cistanche,phelipaea,phelypaea
472,Crepis,zacyntha,zacintha
474,Cressa,eretica,cretica
591,Fontanesia,phillyreoides,philliraeoides


##### Closest match for genus exists
##### but closest match for epithet doesn't

In [37]:
vol1_df[(~(vol1_df['mouterde_epithet'].isna()) & ~(vol1_df['mouterde_epithet'] == "")) & (vol1_df['mout_epithet_close_match'].isna()) & (vol1_df['genus_matched'] == True)][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
26,Aegilotricum,loreti,NaN


In [39]:
vol2_df[~(vol2_df['mouterde_epithet'].isna()) & ~(vol2_df['mouterde_epithet'] == "") & (vol2_df['mout_epithet_close_match'].isna()) & (vol2_df['genus_matched'] == True)][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
393,Callitriche,sp.,NaN
396,Calycotome,villosa,NaN
433,Ceratocephalus,falcafus,NaN
504,Consolida,flâva,NaN
668,Erucaria,boveana,NaN
1070,Linum,corymbulosum(Planch.)Reichenb.,NaN
1075,Linum,sulphureurn,NaN


In [40]:
vol3_df[~(vol3_df['mouterde_epithet'].isna()) & ~(vol3_df['mouterde_epithet'] == "") & (vol3_df['mout_epithet_close_match'].isna()) & (vol3_df['genus_matched'] == True)][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
697,Hormuzakia,aggregata,NaN
904,Myopordon,genitz,NaN
946,Odontites,hitea,NaN
1054,Phyllocara,aucheri,NaN
1114,Psychrogeton,son,NaN


In [41]:
genus = "Hormuzakia"#"Achillea"
epithet = "aggregata"#"aleppica"
wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet)] 
# but it's on ipni? https://www.ipni.org/n/117364-1

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list


### Finding closest infra

In [46]:
#uses default matching criteria of difflib
def difflib_closest_infra_match(row):
    infra = row['mouterde_infra']
    epithet = row['mout_epithet_close_match'] 
    genus = row['mout_genus_close_match']
    if isinstance(infra, str) and row['genus_matched'] and row['mout_epithet_close_match'] and infra != "":
        matching_list = wcvp_df[(wcvp_df['genus'] == genus)  & (wcvp_df['species'] == epithet)]['infraspecies'].unique()
        cleanedList = [x for x in matching_list if str(x) != 'nan']
        match_result = difflib.get_close_matches(infra, cleanedList)  
        
        if match_result:
            return match_result[0]
        else:
            return np.NaN
    else: 
        return np.NaN

In [45]:
startTime = time.time()
#prints the ones with no closest match
print("---- vol1 infra match ----")
vol1_df['mout_infra_close_match'] = vol1_df.apply(difflib_closest_infra_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

startTime = time.time()
print("---- vol2 infra match ----")
vol2_df['mout_infra_close_match'] = vol2_df.apply(difflib_closest_infra_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

startTime = time.time()
print("---- vol3 infra match ----")
vol3_df['mout_infra_close_match'] = vol3_df.apply(difflib_closest_infra_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

---- vol1 infra match ----
Seconds to Run:0.013285160064697266
-------- complete --------

---- vol2 infra match ----
Seconds to Run:0.31200098991394043
-------- complete --------

---- vol3 infra match ----
Seconds to Run:14.632593870162964
-------- complete --------



In [51]:
print(vol3_df[(vol3_df['mout_infra_close_match'] != vol3_df['mouterde_infra'].str.strip()) & ~(vol3_df['mouterde_infra'].isna()) & ~(vol3_df['mouterde_infra'] == "")][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates().shape[0])
vol3_df[(vol3_df['mout_infra_close_match'] != vol3_df['mouterde_infra'].str.strip()) & ~(vol3_df['mouterde_infra'].isna()) &  ~(vol3_df['mouterde_infra'] == "")][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates()

110


,mout_genus_close_match,mout_epithet_close_match,mouterde_infra,mout_infra_close_match
61,Anchusa,strigosa,mesopotamica,NaN
82,Anthemis,cretica,discoidea,NaN
105,Antirrhinum,majus,angustifolium,glutinosum
113,Arctium,minus,minus,NaN
136,Asperula,glomerata,capitata,NaN
...,...,...,...,...
1578,Veronica,anagallis-aquatica,michauxii,NaN
1580,Veronica,anagalloides,anagallis-aquatica,anagalloides
1597,Veronica,macrostachya,orientalis,NaN
1599,Veronica,orientalis,leiocarpa,NaN


In [52]:
# need to fix it a bit :D 
#  print(vol3_df[(vol3_df['mout_infra_close_match'] != vol3_df['mouterde_infra']) & ~(vol3_df['mout_infra_close_match'].isna())][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates().shape[0])
# vol3_df[(vol3_df['mout_infra_close_match'] != vol3_df['mouterde_infra']) & ~(vol3_df['mout_infra_close_match'].isna())][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates()

In [53]:
# print(vol3_df[(vol3_df['mout_infra_close_match'] == vol3_df['mouterde_infra']) & ~(vol3_df['mout_infra_close_match'].isna())][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates().shape[0])
# vol3_df[(vol3_df['mout_infra_close_match'] == vol3_df['mouterde_infra']) & ~(vol3_df['mout_infra_close_match'].isna())][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates()

### merge by genus, epithet, infra 

for now I'm not going to match based on mout_infra_close_match 
just:

mout_genus_close_match and mout_epithet_close_match

(if genus_matched == True)

In [54]:
vol1_df

,Unnamed: 0,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon_rank,taxon_rank_detailed,mouterde_author_list,mout_genus_close_match,genus_matched,mout_epithet_close_match,mout_infra_close_match
0,1,,cilicica,,Ant. et Ky,epithet,epithet,"[Ant., Ky]",NaN,False,NaN,NaN
1,5,Acanthophyllum,,,,genus,genus,,Acanthophyllum,True,NaN,NaN
2,6,Acanthophyllum,kurdicum,,Boiss. et Hausskn.,epithet,epithet,"[Boiss., Hausskn.]",Acanthophyllum,True,kurdicum,NaN
3,23,Achyranthes,,,,genus,genus,,Achyranthes,True,NaN,NaN
4,24,Achyranthes,aspera,,L.,epithet,epithet,[L.],Achyranthes,True,aspera,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1377,5473,Vulpia,uniglumis,,(Sol.) Dumort.,epithet,epithet,"[Sol., Dumort.]",Vulpia,True,uniglumis,NaN
1378,5481,Zannichellia,,,,genus,genus,,Zannichellia,True,NaN,NaN
1379,5482,Zannichellia,palustris,,L.,epithet,epithet,[L.],Zannichellia,True,palustris,NaN
1380,5495,Zostera,,,,genus,genus,,Zostera,True,NaN,NaN


In [61]:
vol1_df

,Unnamed: 0,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon_rank,taxon_rank_detailed,mouterde_author_list,mout_genus_close_match,genus_matched,mout_epithet_close_match,mout_infra_close_match
0,1,,cilicica,,Ant. et Ky,epithet,epithet,"[Ant., Ky]",NaN,False,NaN,NaN
1,5,Acanthophyllum,,,,genus,genus,,Acanthophyllum,True,NaN,NaN
2,6,Acanthophyllum,kurdicum,,Boiss. et Hausskn.,epithet,epithet,"[Boiss., Hausskn.]",Acanthophyllum,True,kurdicum,NaN
3,23,Achyranthes,,,,genus,genus,,Achyranthes,True,NaN,NaN
4,24,Achyranthes,aspera,,L.,epithet,epithet,[L.],Achyranthes,True,aspera,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1377,5473,Vulpia,uniglumis,,(Sol.) Dumort.,epithet,epithet,"[Sol., Dumort.]",Vulpia,True,uniglumis,NaN
1378,5481,Zannichellia,,,,genus,genus,,Zannichellia,True,NaN,NaN
1379,5482,Zannichellia,palustris,,L.,epithet,epithet,[L.],Zannichellia,True,palustris,NaN
1380,5495,Zostera,,,,genus,genus,,Zostera,True,NaN,NaN


In [62]:
vol1_genus_matched_df = vol1_df[vol1_df['genus_matched'] == True].copy()
vol2_genus_matched_df = vol2_df[vol2_df['genus_matched'] == True].copy()
vol3_genus_matched_df = vol3_df[vol3_df['genus_matched'] == True].copy()

vol1_genus_matched_df.replace(np.nan, 'np.NaN', inplace=True)
vol2_genus_matched_df.replace(np.nan, 'np.NaN', inplace=True)
vol3_genus_matched_df.replace(np.nan, 'np.NaN', inplace=True)

vol1_genus_matched_df.replace('', 'np.NaN', inplace=True)
vol2_genus_matched_df.replace('', 'np.NaN', inplace=True)
vol3_genus_matched_df.replace('', 'np.NaN', inplace=True)

wcvp_merge_copy = wcvp_df.copy()
wcvp_merge_copy.replace(np.nan, 'np.NaN', inplace=True)

In [63]:
vol1_merge_df = pd.merge(vol1_genus_matched_df, 
                         wcvp_merge_copy, 
                         how='inner', 
                         left_on=['mout_genus_close_match','mout_epithet_close_match', 'mouterde_infra'], 
                         right_on = ['genus','species', 'infraspecies'])

vol1_merge_simplified = vol1_merge_df[['mouterde_genus',
                                       'mouterde_epithet', 
                                       'mouterde_infra', 
                                       'mouterde_author', 
                                       'mouterde_author_list', 
                                       'author_list', 
                                       'authors', 
                                       'genus',
                                       'species'
                                       ]]

In [64]:
vol2_merge_df = pd.merge(vol2_genus_matched_df, 
                         wcvp_merge_copy, 
                         how='inner', 
                         left_on=['mout_genus_close_match','mout_epithet_close_match', 'mouterde_infra'], 
                         right_on = ['genus','species', 'infraspecies'])

vol2_merge_simplified = vol2_merge_df[['mouterde_genus',
                                       'mouterde_epithet', 
                                       'mouterde_infra', 
                                       'mouterde_author', 
                                       'mouterde_author_list', 
                                       'author_list', 
                                       'authors', 
                                       'genus',
                                       'species'
                                       ]]

In [65]:
vol3_merge_df = pd.merge(vol3_genus_matched_df, 
                         wcvp_merge_copy, 
                         how='inner', 
                         left_on=['mout_genus_close_match','mout_epithet_close_match', 'mouterde_infra'], 
                         right_on = ['genus','species', 'infraspecies'])

vol3_merge_simplified = vol3_merge_df[['mouterde_genus',
                                       'mouterde_epithet', 
                                       'mouterde_infra', 
                                       'mouterde_author', 
                                       'mouterde_author_list', 
                                       'author_list', 
                                       'authors', 
                                       'genus',
                                       'species'
                                       ]]

In [67]:
# not here?vol3_merge_simplified[vol3_merge_simplified["author_match_type"] == "type 2"]

In [68]:
genus = "Achillea"
epithet = "membranacea"
wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet)]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list
149944,174102-1,Asteraceae,Achillea,membranacea,NaN,Achillea membranacea,DC.,SPECIES,Accepted,NaN,NaN,NaN,60436826-2,Achillea,L.,In review,Prodr. 6: 32 (1838),NaN,[DC.]


In [71]:
wcvp_df[(wcvp_df['genus'] == "Calamintha") & (wcvp_df['species'] == "nepeta")]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list
640889,445481-1,Lamiaceae,Calamintha,nepeta,NaN,Calamintha nepeta,Willk.,SPECIES,Synonym,77113937-1,Clinopodium nepeta subsp. spruneri,(Boiss.) Bartolucci & F.Conti,NaN,NaN,NaN,Reviewed,Flora 35: 277 (1852),NaN,[Willk.]
640890,445482-1,Lamiaceae,Calamintha,nepeta,NaN,Calamintha nepeta,(L.) Savi,SPECIES,Homotypic_Synonym,445833-1,Clinopodium nepeta,(L.) Kuntze,NaN,NaN,NaN,Reviewed,Fl. Pis. 1: 197 (1798),450080-1,"[L., Savi]"
640891,883811-1,Lamiaceae,Calamintha,nepeta,glandulosa,Calamintha nepeta subsp. glandulosa,(Req.) P.W.Ball,SUBSPECIES,Synonym,77113937-1,Clinopodium nepeta subsp. spruneri,(Boiss.) Bartolucci & F.Conti,NaN,NaN,NaN,Reviewed,Bot. J. Linn. Soc. 65: 347 (1972),461193-1,"[Req., P.W.Ball]"
640892,77256124-1,Lamiaceae,Calamintha,nepeta,gussonei,Calamintha nepeta subsp. gussonei,Tod. ex Nyman,SUBSPECIES,Synonym,77167928-1,Clinopodium nepeta subsp. nepeta,NaN,NaN,NaN,NaN,Reviewed,Consp. Fl. Eur.: 588 (1881),NaN,[Tod.]
640893,77101862-1,Lamiaceae,Calamintha,nepeta,heterotricha,Calamintha nepeta subsp. heterotricha,(Boiss. & Reut.) Dobignard,SUBSPECIES,Homotypic_Synonym,1009797-1,Clinopodium heterotrichum,(Boiss. & Reut.) Govaerts,NaN,NaN,NaN,Reviewed,J. Bot. Soc. Bot. France 46-47: 57 (2009),445435-1,"[Boiss., Reut., Dobignard]"
640894,77255999-1,Lamiaceae,Calamintha,nepeta,spruneri,Calamintha nepeta subsp. spruneri,(Boiss.) Nyman,SUBSPECIES,Homotypic_Synonym,77113937-1,Clinopodium nepeta subsp. spruneri,(Boiss.) Bartolucci & F.Conti,NaN,NaN,NaN,Reviewed,Consp. Fl. Eur.: 588 (1881),445528-1,"[Boiss., Nyman]"
640895,998869-1,Lamiaceae,Calamintha,nepeta,sylvatica,Calamintha nepeta subsp. sylvatica,(Bromf.) R.Morales,SUBSPECIES,Synonym,77167916-1,Clinopodium menthifolium subsp. menthifolium,NaN,NaN,NaN,NaN,Reviewed,Anales Jard. Bot. Madrid 55: 271 (1997),445538-1,"[Bromf., R.Morales]"


### Matching authors

#### functions for author overlap and types

In [72]:
def get_num_overlap(row):
    if row['mouterde_author_list'] == "np.NaN" or row['author_list'] == "np.NaN":
        return -1

    authors_set_src = row['mouterde_author_list'] #not a set to account for (L.) L. situation
    authors_set_dst = set(row['author_list'])
    num_src = 0
    for author in authors_set_src: 
        _, score = process.extract(author, authors_set_dst)[0]
        if score > 85:
            num_src += 1
    return num_src

In [73]:
def get_match_type(row):
    authors_set_src = row['mouterde_author_list'] #not a set to account for (L.) L. situation
    authors_set_dst = set(row['author_list'])
    num_overlap = row['num_author_overlap']
    
    if row['mouterde_author_list'] == "np.NaN" and row['author_list'] == "np.NaN" :
        return "No author"

    elif row['mouterde_author_list'] == "np.NaN":
        return "No Mout. author"
    
    elif row['author_list'] == "np.NaN":
        return "No WCVP author"
        
    type = "NO TYPE"
    if num_overlap == 0 and len(authors_set_src) > 0 and  len(authors_set_dst) > 0:
        type = "type -1"
    elif num_overlap == len(authors_set_src) and len(authors_set_src) == len(authors_set_dst):
        type = "type 0" #everything the same
    elif num_overlap == len(authors_set_src):
        type = "type 1" #everything in src match but dst set has more 
    elif num_overlap == len(authors_set_dst):
        type = "type 2" #everything in dst match but src has more 

    elif num_overlap < min(len(authors_set_src), len(authors_set_dst)):
        type = "type 3" # a subset of dst or src do not overlap (and one is not a proper subset of another)
    else:
        print(num_overlap, authors_set_src, authors_set_dst)
    return type

In [74]:
vol1_merge_simplified["num_author_overlap"] = vol1_merge_simplified.apply(get_num_overlap, axis=1)
vol1_merge_simplified["author_match_type"] = vol1_merge_simplified.apply(get_match_type, axis=1)

/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_40057/99236302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol1_merge_simplified["num_author_overlap"] = vol1_merge_simplified.apply(get_num_overlap, axis=1)
/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_40057/99236302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol1_merge_simplified["author_match_type"] = vol1_merge_simplified.apply(get_match_type, axis=1)


In [75]:
vol2_merge_simplified["num_author_overlap"] = vol2_merge_simplified.apply(get_num_overlap, axis=1)
vol2_merge_simplified["author_match_type"] = vol2_merge_simplified.apply(get_match_type, axis=1)

/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_40057/1524690916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol2_merge_simplified["num_author_overlap"] = vol2_merge_simplified.apply(get_num_overlap, axis=1)
/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_40057/1524690916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol2_merge_simplified["author_match_type"] = vol2_merge_simplified.apply(get_match_type, axis=1)


In [76]:
vol3_merge_simplified["num_author_overlap"] = vol3_merge_simplified.apply(get_num_overlap, axis=1)
vol3_merge_simplified["author_match_type"] = vol3_merge_simplified.apply(get_match_type, axis=1)

2 ['Sm.', 'Sibth.', 'Sm.'] {'Sm.'}
2 ['Boiss.', 'Reut.', ' Boiss.'] {'Boiss.'}
2 ['Sm.', 'Sibth.', 'Sm. '] {'Sm.'}


/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_40057/1213449415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol3_merge_simplified["num_author_overlap"] = vol3_merge_simplified.apply(get_num_overlap, axis=1)
/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_40057/1213449415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol3_merge_simplified["author_match_type"] = vol3_merge_simplified.apply(get_match_type, axis=1)


2 {'Reut.', 'Boiss.', ' Boiss.'} {'Boiss.'} -> goes to else because of this ... NO TYPE no longer needed :)

#### group rows with overlapping genus, epithet, infra, author by best author match with wcvp


In [77]:
#vol1_merge_simplified = vol1_merge_simplified.loc[vol1_merge_simplified.groupby(['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]#[['bookid', 'conceptid', 'weight']]
vol1_merge_simplified_grouped = vol1_merge_simplified.loc[vol1_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]
vol2_merge_simplified_grouped = vol2_merge_simplified.loc[vol2_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]
vol3_merge_simplified_grouped = vol3_merge_simplified.loc[vol3_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]

In [78]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped['genus'] == "Calamintha") & (vol3_merge_simplified_grouped['species'] == "nepeta")]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
206,Calamintha,nepeta,np.NaN,(L.) Savi,"[L., Savi]","[L., Savi]",(L.) Savi,Calamintha,nepeta,2,type 0


In [79]:
vol3_merge_simplified_grouped[vol3_merge_simplified_grouped['author_match_type'] == "type 2"]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
21,Achillea,membranacea,np.NaN,(Labili.). DC.,"[Labili., . DC.]",[DC.],DC.,Achillea,membranacea,1,type 2
150,Asperula,rivalis,np.NaN,Sibth. et Sm.,"[Sibth., Sm.]",[Sm.],Sm.,Asperula,rivalis,1,type 2
204,Calamintha,incana,np.NaN,(Sibth. et Smith) Boiss.,"[Sibth., Smith, Boiss.]","[Sm., Boiss.]",(Sm.) Boiss.,Calamintha,incana,2,type 2
238,Campanula,fastigiata,np.NaN,Léon Dufour in A. DC.,"[Léon Dufour, A. DC.]",[Dufour],Dufour ex Schult.,Campanula,fastigiata,1,type 2
323,Centaurea,cyanoides,np.NaN,Berg et Wahlenb.,"[Berg, Wahlenb. ]",[Wahlenb.],Wahlenb.,Centaurea,cyanoides,1,type 2
328,Centaurea,drabifolia,np.NaN,Sibth. et Sm.,"[Sibth., Sm.]",[Sm.],Sm.,Centaurea,drabifolia,1,type 2
343,Centaurea,microcnicus,np.NaN,Reese et Sam. in Rech. fil.,"[Reese, Sam., Rech. fil.]","[Reese, Sam.]",Reese & Sam.,Centaurea,microcnicus,2,type 2
412,Cicerbita,mulgedioides,np.NaN,(Schultz Bip. ex Vis. et Pané.) Beauverd,"[Schultz Bip., Beauverd ]",[Beauverd],Beauverd,Cicerbita,mulgedioides,1,type 2
476,Crépis,aculeata,np.NaN,(DC.) Boiss.,"[DC., Boiss.]",[Boiss.],Boiss.,Crepis,aculeata,1,type 2
512,Cruciata,coronata,np.NaN,(Sibth. et Sm.) Ehrendorfer,"[Sibth., Sm., Ehrendorfer]","[Sm., Ehrend.]",(Sm.) Ehrend.,Cruciata,coronata,2,type 2


In [80]:
vol3_merge_simplified_grouped[vol3_merge_simplified_grouped['author_match_type'] == "type -1"].head(50)

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
23,Achillea,wilhelmsii,np.NaN,C. Koch,[C. Koch],[K.Koch],K.Koch,Achillea,wilhelmsii,0,type -1
64,Anchusa,milleri,np.NaN,Willd.,[Willd.],[Lam.],Lam. ex Spreng.,Anchusa,milleri,0,type -1
65,Anchusa,strigosa,np.NaN,Labili.,[Labili.],[Labill.],Labill.,Anchusa,strigosa,0,type -1
110,Anthémis,tinctoria,discoidea,(AH.) Vahl,"[AH., Vahl]","[All., DC.]",(All.) DC.,Anthemis,tinctoria,0,type -1
153,Asteriscus,np.NaN,np.NaN,Miller,[Miller],[Moench],Moench,Asteriscus,np.NaN,0,type -1
168,Bellardia,np.NaN,np.NaN,Ail.,[Ail.],[Colla],Colla,Bellardia,np.NaN,0,type -1
198,Bungea,np.NaN,np.NaN,CA. Meyer,[CA. Meyer],[C.A.Mey.],C.A.Mey.,Bungea,np.NaN,0,type -1
202,Calamintha,np.NaN,np.NaN,Moench,[Moench],[Mill.],Mill.,Calamintha,np.NaN,0,type -1
235,Campanula,damascena,np.NaN,Labili.,[Labili.],[Labill.],Labill.,Campanula,damascena,0,type -1
250,Campanula,retrorsa,np.NaN,Labili.,[Labili. ],[Labill.],Labill.,Campanula,retrorsa,0,type -1


#### Playing around with different author matchings

##### Type -1 (no match but authors available)

In [81]:
vol1_merge_simplified_grouped[vol1_merge_simplified_grouped['author_match_type'] == 'type -1']

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
29,Aegilotricum,loreti,np.NaN,(Richt.) Maire et Weiller,"[Richt., Maire, Weiller]",[R.Wagner],R.Wagner ex Tscherm.-Seys.,Aegilotricum,np.NaN,0,type -1
48,Alisma,gramineum,np.NaN,Gmel.,[Gmel.],[Lej.],Lej.,Alisma,gramineum,0,type -1
49,Alisma,graminifolium,np.NaN,Ehrh.,[Ehrh. ],[Glück],Glück,Alisma,graminifolium,0,type -1
50,Alisma,lanceolatum,np.NaN,Whit.,[Whit.],[With.],With.,Alisma,lanceolatum,0,type -1
60,Allium,aschersonianum,np.NaN,W. Barb.,[W. Barb. ],[Barbey],Barbey,Allium,aschersonianum,0,type -1
...,...,...,...,...,...,...,...,...,...,...,...
1281,Tulipa,agenensis,np.NaN,D.C.,[D.C.],[Redouté],Redouté,Tulipa,agenensis,0,type -1
1282,Tulipa,aleppensis,np.NaN,Regel,[Regel],[Boiss.],Boiss. ex Regel,Tulipa,aleppensis,0,type -1
1284,Tulipa,biflora,np.NaN,Pali.,[Pali.],[Pall.],Pall.,Tulipa,biflora,0,type -1
1286,Tulipa,montana,np.NaN,Lindley,[Lindley],[Lindl.],Lindl.,Tulipa,montana,0,type -1


In [82]:
vol2_merge_simplified_grouped[vol2_merge_simplified_grouped['author_match_type'] == 'type -1']

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
8,Acer,tauricolum,np.NaN,Boiss. et Bal.,"[Boiss., Bal.]",[Dippel],Dippel,Acer,tauricum,0,type -1
24,Adonis,microcarpa,np.NaN,D.C.,[D.C.],[DC.],DC.,Adonis,microcarpa,0,type -1
34,Aethionema,coridifolium,np.NaN,D.C.,[D.C.],[DC.],DC.,Aethionema,coridifolium,0,type -1
37,Aethionema,oppositifolium,np.NaN,(Lab.) Boiss.,"[Lab., Boiss.]","[Pers., Hedge]",(Pers.) Hedge,Aethionema,oppositifolium,0,type -1
67,Alhagi,maurorum,np.NaN,Willd.,[Willd.],[Medik.],Medik.,Alhagi,maurorum,0,type -1
...,...,...,...,...,...,...,...,...,...,...,...
1493,Ranunculus,cornutus,np.NaN,D.C.,[D.C.],[Boiss.],Boiss.,Ranunculus,cornutus,0,type -1
1499,Ranunculus,demissus,np.NaN,D.C.,[D.C.],[DC.],DC.,Ranunculus,demissus,0,type -1
1501,Ranunculus,lateriflorus,np.NaN,D.C.,[D.C.],[DC.],DC.,Ranunculus,lateriflorus,0,type -1
1511,Ranunculus,orbiculatus,np.NaN,Boiss. et Bo.,"[Boiss., Bo.]",[Blanche],Blanche,Ranunculus,orbiculatus,0,type -1


In [83]:
vol3_merge_simplified_grouped[vol3_merge_simplified_grouped['author_match_type'] == 'type -1']

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
23,Achillea,wilhelmsii,np.NaN,C. Koch,[C. Koch],[K.Koch],K.Koch,Achillea,wilhelmsii,0,type -1
64,Anchusa,milleri,np.NaN,Willd.,[Willd.],[Lam.],Lam. ex Spreng.,Anchusa,milleri,0,type -1
65,Anchusa,strigosa,np.NaN,Labili.,[Labili.],[Labill.],Labill.,Anchusa,strigosa,0,type -1
110,Anthémis,tinctoria,discoidea,(AH.) Vahl,"[AH., Vahl]","[All., DC.]",(All.) DC.,Anthemis,tinctoria,0,type -1
153,Asteriscus,np.NaN,np.NaN,Miller,[Miller],[Moench],Moench,Asteriscus,np.NaN,0,type -1
...,...,...,...,...,...,...,...,...,...,...,...
1562,Vaillantia,np.NaN,np.NaN,auct. mult. non L.,"[auct., L.]",[Hoffm.],Hoffm.,Vaillantia,np.NaN,0,type -1
1578,Valerianella,cymbicarpa,np.NaN,G.A. Meyer,[G.A. Meyer ],[C.A.Mey.],C.A.Mey.,Valerianella,cymbaecarpa,0,type -1
1627,Verbascum,oreophilum,np.NaN,C. Koch,[C. Koch],[K.Koch],K.Koch,Verbascum,oreophilum,0,type -1
1634,Verbascum,ported,np.NaN,Post,[Post],[Willk.],Willk.,Verbascum,portae,0,type -1


In [85]:
print(vol1_merge_simplified[vol1_merge_simplified['author_match_type'] == 'type -1'].shape[0],
      vol2_merge_simplified[vol2_merge_simplified['author_match_type'] == 'type -1'].shape[0],
      vol3_merge_simplified[vol3_merge_simplified['author_match_type'] == 'type -1'].shape[0])

232 254 271


In [86]:
genus = "Anchusa"#"Achillea"
epithet = "milleri"#"aleppica"
infra = "np.NaN"#"aleppica"
vol3_merge_simplified[(vol3_merge_simplified['mouterde_genus'] == genus) & (vol3_merge_simplified['mouterde_epithet'] == epithet) & (vol3_merge_simplified['mouterde_infra'] == infra)]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
64,Anchusa,milleri,np.NaN,Willd.,[Willd.],[Lam.],Lam. ex Spreng.,Anchusa,milleri,0,type -1


In [87]:
genus = "Anchusa"#"Achillea"
epithet = "milleri"#"aleppica"
infra = "np.NaN"#"aleppica"
wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet) & (wcvp_df['infraspecies'].isna())]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list
288711,927538-1,Boraginaceae,Anchusa,milleri,NaN,Anchusa milleri,Lam. ex Spreng.,SPECIES,Accepted,NaN,NaN,NaN,4451-1,Anchusa,L.,In review,"Bot. Gart. Halle, Nachtr. 1: 11 (1801)",NaN,[Lam.]


##### untitled

In [88]:
vol3_merge_simplified[(vol3_merge_simplified["author_match_type"] == "No author") |
                      (vol3_merge_simplified["author_match_type"] == "No Mout. author") |
                      (vol3_merge_simplified["author_match_type"] == 'No WCVP author') 
                     ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Achillea,aleppica,-1,No author
285,Carlina,oligocephala,oligocephala,np.NaN,np.NaN,np.NaN,np.NaN,Carlina,oligocephala,-1,No author
295,Carthamus,tenuis,tenuis,np.NaN,np.NaN,np.NaN,np.NaN,Carthamus,tenuis,-1,No author
483,Crépis,foetida,foetida,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,foetida,-1,No author
495,Crépis,sancta,sancta,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,sancta,-1,No author
671,Galium,thiebautii,np.NaN,np.NaN,np.NaN,[Ehrend.],Ehrend.,Galium,thiebautii,-1,No Mout. author
737,Hyoscyamus,albu^L.,np.NaN,np.NaN,np.NaN,[L.],L.,Hyoscyamus,albus,-1,No Mout. author
781,Knautia,integrifolia,integrifolia,np.NaN,np.NaN,np.NaN,np.NaN,Knautia,integrifolia,-1,No author
955,Myopordon,genitz,np.NaN,np.NaN,np.NaN,[Boiss.],Boiss.,Myopordon,np.NaN,-1,No Mout. author
1038,Origanum,bargyli,np.NaN,np.NaN,np.NaN,[Mouterde],Mouterde,Origanum,bargyli,-1,No Mout. author


In [89]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped["author_match_type"] == "No author") |
                              (vol3_merge_simplified_grouped["author_match_type"] == "No Mout. author") |
                              (vol3_merge_simplified_grouped["author_match_type"] == 'No WCVP author') 
                             ].shape

(22, 11)

In [90]:
vol3_merge_simplified_grouped["author_match_type"].unique()

array(['type 0', 'type 3', 'No author', 'type 2', 'type -1', 'type 1',
       'NO TYPE', 'No Mout. author'], dtype=object)

In [91]:
vol1_merge_simplified["author_match_type"].unique()

array(['No Mout. author', 'type 0', 'type -1', 'type 1', 'type 3',
       'type 2'], dtype=object)

In [92]:
vol1_merge_simplified

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,"[Hook., Arn.]",Hook. & Arn.,Acanthophyllum,np.NaN,-1,No Mout. author
1,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,[C.A.Mey.],C.A.Mey.,Acanthophyllum,np.NaN,-1,No Mout. author
2,Acanthophyllum,kurdicum,np.NaN,Boiss. et Hausskn.,"[Boiss., Hausskn.]","[Boiss., Hausskn.]",Boiss. & Hausskn.,Acanthophyllum,kurdicum,2,type 0
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
4,Achyranthes,aspera,np.NaN,L.,[L.],[L.],L.,Achyranthes,aspera,1,type 0
...,...,...,...,...,...,...,...,...,...,...,...
1329,Vulpia,uniglumis,np.NaN,(Sol.) Dumort.,"[Sol., Dumort.]",[Dumort.],Dumort.,Vulpia,uniglumis,1,type 2
1330,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author
1331,Zannichellia,palustris,np.NaN,L.,[L.],[L.],L.,Zannichellia,palustris,1,type 0
1332,Zostera,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Zostera,np.NaN,-1,No Mout. author


In [93]:
vol1_merge_simplified_grouped

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,"[Hook., Arn.]",Hook. & Arn.,Acanthophyllum,np.NaN,-1,No Mout. author
2,Acanthophyllum,kurdicum,np.NaN,Boiss. et Hausskn.,"[Boiss., Hausskn.]","[Boiss., Hausskn.]",Boiss. & Hausskn.,Acanthophyllum,kurdicum,2,type 0
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
4,Achyranthes,aspera,np.NaN,L.,[L.],[L.],L.,Achyranthes,aspera,1,type 0
5,Adiantum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adiantum,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1329,Vulpia,uniglumis,np.NaN,(Sol.) Dumort.,"[Sol., Dumort.]",[Dumort.],Dumort.,Vulpia,uniglumis,1,type 2
1330,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author
1331,Zannichellia,palustris,np.NaN,L.,[L.],[L.],L.,Zannichellia,palustris,1,type 0
1332,Zostera,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Zostera,np.NaN,-1,No Mout. author


In [94]:
vol1_merge_simplified_grouped[vol1_merge_simplified_grouped['author_match_type'] == 'type -1']

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
29,Aegilotricum,loreti,np.NaN,(Richt.) Maire et Weiller,"[Richt., Maire, Weiller]",[R.Wagner],R.Wagner ex Tscherm.-Seys.,Aegilotricum,np.NaN,0,type -1
48,Alisma,gramineum,np.NaN,Gmel.,[Gmel.],[Lej.],Lej.,Alisma,gramineum,0,type -1
49,Alisma,graminifolium,np.NaN,Ehrh.,[Ehrh. ],[Glück],Glück,Alisma,graminifolium,0,type -1
50,Alisma,lanceolatum,np.NaN,Whit.,[Whit.],[With.],With.,Alisma,lanceolatum,0,type -1
60,Allium,aschersonianum,np.NaN,W. Barb.,[W. Barb. ],[Barbey],Barbey,Allium,aschersonianum,0,type -1
...,...,...,...,...,...,...,...,...,...,...,...
1281,Tulipa,agenensis,np.NaN,D.C.,[D.C.],[Redouté],Redouté,Tulipa,agenensis,0,type -1
1282,Tulipa,aleppensis,np.NaN,Regel,[Regel],[Boiss.],Boiss. ex Regel,Tulipa,aleppensis,0,type -1
1284,Tulipa,biflora,np.NaN,Pali.,[Pali.],[Pall.],Pall.,Tulipa,biflora,0,type -1
1286,Tulipa,montana,np.NaN,Lindley,[Lindley],[Lindl.],Lindl.,Tulipa,montana,0,type -1


In [95]:
genus = "Alisma"
epithet = "gramineum"
infra = np.NaN
wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet)]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list
22305,77097302-1,Alismataceae,Alisma,gramineum,NaN,Alisma gramineum,Lej.,SPECIES,Accepted,NaN,NaN,NaN,328557-2,Alisma,L.,Reviewed,Fl. Spa 1: 175 (1811),NaN,[Lej.]
22306,874063-1,Alismataceae,Alisma,gramineum,aestuosum,Alisma gramineum f. aestuosum,(Bolle) Soó,Form,Synonym,77097302-1,Alisma gramineum,Lej.,NaN,NaN,NaN,Reviewed,Acta Bot. Acad. Sci. Hung. 16: 365 (1971),NaN,"[Bolle, Soó]"
22307,8479-2,Alismataceae,Alisma,gramineum,angustissimum,Alisma gramineum var. angustissimum,(DC.) Hendricks,VARIETY,Synonym,77097302-1,Alisma gramineum,Lej.,NaN,NaN,NaN,Reviewed,Amer. Midl. Naturalist 58: 489 (1957),8491-2,"[DC., Hendricks]"
22308,8480-2,Alismataceae,Alisma,gramineum,geyeri,Alisma gramineum var. geyeri,(Torr.) Sam.,VARIETY,Synonym,77097302-1,Alisma gramineum,Lej.,NaN,NaN,NaN,Reviewed,Ark. Bot. 24A(7): 43 (1932),58204-1,"[Torr., Sam.]"
22309,8482-2,Alismataceae,Alisma,gramineum,graminifolium,Alisma gramineum var. graminifolium,(Wahlenb.) Hendricks,VARIETY,Synonym,60449258-2,Alisma wahlenbergii,(Holmb.) Juz.,NaN,NaN,NaN,Reviewed,Amer. Midl. Naturalist 58: 487 (1957),8494-2,"[Wahlenb., Hendricks]"
22310,1160212-2,Alismataceae,Alisma,gramineum,wahlenbergii,Alisma gramineum subsp. wahlenbergii,Holmb.,SUBSPECIES,Homotypic_Synonym,60449258-2,Alisma wahlenbergii,(Holmb.) Juz.,NaN,NaN,NaN,Reviewed,Bot. Not. 1922: 207 (1922),NaN,[Holmb.]
22311,8483-2,Alismataceae,Alisma,gramineum,wahlenbergii,Alisma gramineum var. wahlenbergii,(Holmb.) Raymond & Kucyn.,VARIETY,Homotypic_Synonym,60449258-2,Alisma wahlenbergii,(Holmb.) Juz.,NaN,NaN,NaN,Reviewed,Rhodora 50: 179 (1948),1160212-2,"[Holmb., Raymond, Kucyn.]"


##### Vol1 No Author

In [96]:
vol1_merge_simplified_grouped[(vol1_merge_simplified_grouped["author_match_type"] == "No author") |
                              (vol1_merge_simplified_grouped["author_match_type"] == "No Mout. author") |
                              (vol1_merge_simplified_grouped["author_match_type"] == 'No WCVP author') 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,"[Hook., Arn.]",Hook. & Arn.,Acanthophyllum,np.NaN,-1,No Mout. author
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
5,Adiantum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adiantum,np.NaN,-1,No Mout. author
7,Aegilops,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Aegilops,np.NaN,-1,No Mout. author
28,Aegilotricum,np.NaN,np.NaN,np.NaN,np.NaN,[R.Wagner],R.Wagner ex Tscherm.-Seys.,Aegilotricum,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1315,Velezia,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Velezia,np.NaN,-1,No Mout. author
1318,Viscum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Viscum,np.NaN,-1,No Mout. author
1321,Vulpia,np.NaN,np.NaN,np.NaN,np.NaN,[C.C.Gmel.],C.C.Gmel.,Vulpia,np.NaN,-1,No Mout. author
1330,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author


In [97]:
vol1_merge_simplified_grouped[(vol1_merge_simplified_grouped["author_match_type"] == "No author") 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type


In [98]:
vol1_merge_simplified_grouped[(vol1_merge_simplified_grouped["author_match_type"] == 'No WCVP author')]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type


In [99]:
vol1_merge_simplified_grouped[(vol1_merge_simplified_grouped["author_match_type"] == "No Mout. author")]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,"[Hook., Arn.]",Hook. & Arn.,Acanthophyllum,np.NaN,-1,No Mout. author
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
5,Adiantum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adiantum,np.NaN,-1,No Mout. author
7,Aegilops,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Aegilops,np.NaN,-1,No Mout. author
28,Aegilotricum,np.NaN,np.NaN,np.NaN,np.NaN,[R.Wagner],R.Wagner ex Tscherm.-Seys.,Aegilotricum,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1315,Velezia,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Velezia,np.NaN,-1,No Mout. author
1318,Viscum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Viscum,np.NaN,-1,No Mout. author
1321,Vulpia,np.NaN,np.NaN,np.NaN,np.NaN,[C.C.Gmel.],C.C.Gmel.,Vulpia,np.NaN,-1,No Mout. author
1330,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author


In [100]:
vol1_merge_simplified[(vol1_merge_simplified["author_match_type"] == "No author") |
                              (vol1_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol1_merge_simplified["author_match_type"] == 'No WCVP author') 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,"[Hook., Arn.]",Hook. & Arn.,Acanthophyllum,np.NaN,-1,No Mout. author
1,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,[C.A.Mey.],C.A.Mey.,Acanthophyllum,np.NaN,-1,No Mout. author
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
5,Adiantum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adiantum,np.NaN,-1,No Mout. author
7,Aegilops,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Aegilops,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1315,Velezia,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Velezia,np.NaN,-1,No Mout. author
1318,Viscum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Viscum,np.NaN,-1,No Mout. author
1321,Vulpia,np.NaN,np.NaN,np.NaN,np.NaN,[C.C.Gmel.],C.C.Gmel.,Vulpia,np.NaN,-1,No Mout. author
1330,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author


##### Vol2 No Author

In [101]:
vol2_merge_simplified_grouped[(vol2_merge_simplified_grouped["author_match_type"] == "No author") |
                              (vol2_merge_simplified_grouped["author_match_type"] == "No Mout. author") |
                              (vol2_merge_simplified_grouped["author_match_type"] == 'No WCVP author') 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acacia,np.NaN,np.NaN,np.NaN,np.NaN,[Mill.],Mill.,Acacia,np.NaN,-1,No Mout. author
5,Acer,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Acer,np.NaN,-1,No Mout. author
9,Actinolema,np.NaN,np.NaN,np.NaN,np.NaN,[Fenzl],Fenzl,Actinolema,np.NaN,-1,No Mout. author
12,Adenocarpus,np.NaN,np.NaN,np.NaN,np.NaN,[DC.],DC.,Adenocarpus,np.NaN,-1,No Mout. author
14,Adonis,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adonis,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1478,Punica,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,np.NaN,-1,No Mout. author
1479,Punica,granatum,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,granatum,-1,No Mout. author
1480,Pyracantha,np.NaN,np.NaN,np.NaN,np.NaN,[M.Roem.],M.Roem.,Pyracantha,np.NaN,-1,No Mout. author
1482,Radiola,np.NaN,np.NaN,np.NaN,np.NaN,[Hill],Hill,Radiola,np.NaN,-1,No Mout. author


In [102]:
vol2_merge_simplified_grouped[(vol2_merge_simplified_grouped["author_match_type"] == "No author") 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type


In [103]:
vol2_merge_simplified_grouped[(vol2_merge_simplified_grouped["author_match_type"] == 'No WCVP author')]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type


In [104]:
vol2_merge_simplified_grouped[(vol2_merge_simplified_grouped["author_match_type"] == "No Mout. author")]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acacia,np.NaN,np.NaN,np.NaN,np.NaN,[Mill.],Mill.,Acacia,np.NaN,-1,No Mout. author
5,Acer,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Acer,np.NaN,-1,No Mout. author
9,Actinolema,np.NaN,np.NaN,np.NaN,np.NaN,[Fenzl],Fenzl,Actinolema,np.NaN,-1,No Mout. author
12,Adenocarpus,np.NaN,np.NaN,np.NaN,np.NaN,[DC.],DC.,Adenocarpus,np.NaN,-1,No Mout. author
14,Adonis,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adonis,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1478,Punica,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,np.NaN,-1,No Mout. author
1479,Punica,granatum,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,granatum,-1,No Mout. author
1480,Pyracantha,np.NaN,np.NaN,np.NaN,np.NaN,[M.Roem.],M.Roem.,Pyracantha,np.NaN,-1,No Mout. author
1482,Radiola,np.NaN,np.NaN,np.NaN,np.NaN,[Hill],Hill,Radiola,np.NaN,-1,No Mout. author


In [105]:
vol2_merge_simplified[(vol2_merge_simplified["author_match_type"] == "No author") |
                              (vol2_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol2_merge_simplified["author_match_type"] == 'No WCVP author') 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acacia,np.NaN,np.NaN,np.NaN,np.NaN,[Mill.],Mill.,Acacia,np.NaN,-1,No Mout. author
5,Acer,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Acer,np.NaN,-1,No Mout. author
9,Actinolema,np.NaN,np.NaN,np.NaN,np.NaN,[Fenzl],Fenzl,Actinolema,np.NaN,-1,No Mout. author
12,Adenocarpus,np.NaN,np.NaN,np.NaN,np.NaN,[DC.],DC.,Adenocarpus,np.NaN,-1,No Mout. author
14,Adonis,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adonis,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1478,Punica,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,np.NaN,-1,No Mout. author
1479,Punica,granatum,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,granatum,-1,No Mout. author
1480,Pyracantha,np.NaN,np.NaN,np.NaN,np.NaN,[M.Roem.],M.Roem.,Pyracantha,np.NaN,-1,No Mout. author
1482,Radiola,np.NaN,np.NaN,np.NaN,np.NaN,[Hill],Hill,Radiola,np.NaN,-1,No Mout. author


##### Vol3 No Author

In [107]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped["author_match_type"] == "No author") |
                              (vol3_merge_simplified_grouped["author_match_type"] == "No Mout. author") |
                              (vol3_merge_simplified_grouped["author_match_type"] == 'No WCVP author') 
                             ] #shape[0] 38

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Achillea,aleppica,-1,No author
285,Carlina,oligocephala,oligocephala,np.NaN,np.NaN,np.NaN,np.NaN,Carlina,oligocephala,-1,No author
295,Carthamus,tenuis,tenuis,np.NaN,np.NaN,np.NaN,np.NaN,Carthamus,tenuis,-1,No author
483,Crépis,foetida,foetida,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,foetida,-1,No author
495,Crépis,sancta,sancta,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,sancta,-1,No author
671,Galium,thiebautii,np.NaN,np.NaN,np.NaN,[Ehrend.],Ehrend.,Galium,thiebautii,-1,No Mout. author
737,Hyoscyamus,albu^L.,np.NaN,np.NaN,np.NaN,[L.],L.,Hyoscyamus,albus,-1,No Mout. author
781,Knautia,integrifolia,integrifolia,np.NaN,np.NaN,np.NaN,np.NaN,Knautia,integrifolia,-1,No author
955,Myopordon,genitz,np.NaN,np.NaN,np.NaN,[Boiss.],Boiss.,Myopordon,np.NaN,-1,No Mout. author
1038,Origanum,bargyli,np.NaN,np.NaN,np.NaN,[Mouterde],Mouterde,Origanum,bargyli,-1,No Mout. author


In [108]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped["author_match_type"] == "No author")]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Achillea,aleppica,-1,No author
285,Carlina,oligocephala,oligocephala,np.NaN,np.NaN,np.NaN,np.NaN,Carlina,oligocephala,-1,No author
295,Carthamus,tenuis,tenuis,np.NaN,np.NaN,np.NaN,np.NaN,Carthamus,tenuis,-1,No author
483,Crépis,foetida,foetida,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,foetida,-1,No author
495,Crépis,sancta,sancta,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,sancta,-1,No author
781,Knautia,integrifolia,integrifolia,np.NaN,np.NaN,np.NaN,np.NaN,Knautia,integrifolia,-1,No author
1326,Scrophularia,libanotica,libanotica,np.NaN,np.NaN,np.NaN,np.NaN,Scrophularia,libanotica,-1,No author
1360,Senecio,doriiformis,doriiformis,np.NaN,np.NaN,np.NaN,np.NaN,Senecio,doriiformis,-1,No author
1506,Teucrium,montbretii,montbretii,np.NaN,np.NaN,np.NaN,np.NaN,Teucrium,montbretii,-1,No author
1669,Veronica,panormitana,panormitana,np.NaN,np.NaN,np.NaN,np.NaN,Veronica,panormitana,-1,No author


In [109]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped["author_match_type"] == 'No WCVP author')]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type


In [110]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped["author_match_type"] == "No Mout. author")]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
671,Galium,thiebautii,np.NaN,np.NaN,np.NaN,[Ehrend.],Ehrend.,Galium,thiebautii,-1,No Mout. author
737,Hyoscyamus,albu^L.,np.NaN,np.NaN,np.NaN,[L.],L.,Hyoscyamus,albus,-1,No Mout. author
955,Myopordon,genitz,np.NaN,np.NaN,np.NaN,[Boiss.],Boiss.,Myopordon,np.NaN,-1,No Mout. author
1038,Origanum,bargyli,np.NaN,np.NaN,np.NaN,[Mouterde],Mouterde,Origanum,bargyli,-1,No Mout. author
1040,Origanum,ehrenbergii,np.NaN,np.NaN,np.NaN,[Boiss.],Boiss.,Origanum,ehrenbergii,-1,No Mout. author
1041,Origanum,laevigatum,np.NaN,np.NaN,np.NaN,[Boiss.],Boiss.,Origanum,laevigatum,-1,No Mout. author
1043,Origanum,libanoticum,np.NaN,np.NaN,np.NaN,[Boiss.],Boiss.,Origanum,libanoticum,-1,No Mout. author
1118,Phyteuma,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Phyteuma,np.NaN,-1,No Mout. author
1180,Psychrogeton,son,np.NaN,np.NaN,np.NaN,[Boiss.],Boiss.,Psychrogeton,np.NaN,-1,No Mout. author
1410,SolidagoL.,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Solidago,np.NaN,-1,No Mout. author


In [111]:
vol3_merge_simplified[(vol3_merge_simplified["author_match_type"] == "No author") |
                              (vol3_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol3_merge_simplified["author_match_type"] == 'No WCVP author') ] #shape[0] 48

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Achillea,aleppica,-1,No author
285,Carlina,oligocephala,oligocephala,np.NaN,np.NaN,np.NaN,np.NaN,Carlina,oligocephala,-1,No author
295,Carthamus,tenuis,tenuis,np.NaN,np.NaN,np.NaN,np.NaN,Carthamus,tenuis,-1,No author
483,Crépis,foetida,foetida,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,foetida,-1,No author
495,Crépis,sancta,sancta,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,sancta,-1,No author
671,Galium,thiebautii,np.NaN,np.NaN,np.NaN,[Ehrend.],Ehrend.,Galium,thiebautii,-1,No Mout. author
737,Hyoscyamus,albu^L.,np.NaN,np.NaN,np.NaN,[L.],L.,Hyoscyamus,albus,-1,No Mout. author
781,Knautia,integrifolia,integrifolia,np.NaN,np.NaN,np.NaN,np.NaN,Knautia,integrifolia,-1,No author
955,Myopordon,genitz,np.NaN,np.NaN,np.NaN,[Boiss.],Boiss.,Myopordon,np.NaN,-1,No Mout. author
1038,Origanum,bargyli,np.NaN,np.NaN,np.NaN,[Mouterde],Mouterde,Origanum,bargyli,-1,No Mout. author


##### # NOTE: for all the no Mout. Author types the author may be wrong have and have multiple possibilities!!! 

but that's not the case for the majority fo them which is good! 


In [112]:
#vol1_merge_simplified = vol1_merge_simplified.loc[vol1_merge_simplified.groupby(['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]#[['bookid', 'conceptid', 'weight']]
vol1_merge_simplified_grouped = vol1_merge_simplified.loc[vol1_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]
vol2_merge_simplified_grouped = vol2_merge_simplified.loc[vol2_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]
vol3_merge_simplified_grouped = vol3_merge_simplified.loc[vol3_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]

In [113]:
vol1_no_author_combos = vol1_merge_simplified[(vol1_merge_simplified["author_match_type"] == "No author") |
                              (vol1_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol1_merge_simplified["author_match_type"] == 'No WCVP author')]

vol1_no_author_combos.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author']).filter(lambda x  : len(x) == 1)


,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
5,Adiantum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adiantum,np.NaN,-1,No Mout. author
7,Aegilops,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Aegilops,np.NaN,-1,No Mout. author
28,Aegilotricum,np.NaN,np.NaN,np.NaN,np.NaN,[R.Wagner],R.Wagner ex Tscherm.-Seys.,Aegilotricum,np.NaN,-1,No Mout. author
30,Aellenia,np.NaN,np.NaN,np.NaN,np.NaN,[Ulbr.],Ulbr.,Aellenia,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1315,Velezia,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Velezia,np.NaN,-1,No Mout. author
1318,Viscum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Viscum,np.NaN,-1,No Mout. author
1321,Vulpia,np.NaN,np.NaN,np.NaN,np.NaN,[C.C.Gmel.],C.C.Gmel.,Vulpia,np.NaN,-1,No Mout. author
1330,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author


In [114]:
vol1_no_author_combos.shape[0]

284

In [115]:
vol2_no_author_combos = vol2_merge_simplified[(vol2_merge_simplified["author_match_type"] == "No author") |
                              (vol2_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol2_merge_simplified["author_match_type"] == 'No WCVP author')]

vol2_no_author_combos.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author']).filter(lambda x  : len(x) == 1)


,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acacia,np.NaN,np.NaN,np.NaN,np.NaN,[Mill.],Mill.,Acacia,np.NaN,-1,No Mout. author
5,Acer,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Acer,np.NaN,-1,No Mout. author
9,Actinolema,np.NaN,np.NaN,np.NaN,np.NaN,[Fenzl],Fenzl,Actinolema,np.NaN,-1,No Mout. author
12,Adenocarpus,np.NaN,np.NaN,np.NaN,np.NaN,[DC.],DC.,Adenocarpus,np.NaN,-1,No Mout. author
14,Adonis,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adonis,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1478,Punica,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,np.NaN,-1,No Mout. author
1479,Punica,granatum,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,granatum,-1,No Mout. author
1480,Pyracantha,np.NaN,np.NaN,np.NaN,np.NaN,[M.Roem.],M.Roem.,Pyracantha,np.NaN,-1,No Mout. author
1482,Radiola,np.NaN,np.NaN,np.NaN,np.NaN,[Hill],Hill,Radiola,np.NaN,-1,No Mout. author


In [116]:
vol2_no_author_combos.shape[0]

310

In [117]:
vol3_no_author_combos = vol3_merge_simplified[(vol3_merge_simplified["author_match_type"] == "No author") |
                              (vol3_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol3_merge_simplified["author_match_type"] == 'No WCVP author')]

vol3_no_author_combos.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author']).filter(lambda x  : len(x) == 1) #.shape[0] 34


,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Achillea,aleppica,-1,No author
285,Carlina,oligocephala,oligocephala,np.NaN,np.NaN,np.NaN,np.NaN,Carlina,oligocephala,-1,No author
295,Carthamus,tenuis,tenuis,np.NaN,np.NaN,np.NaN,np.NaN,Carthamus,tenuis,-1,No author
483,Crépis,foetida,foetida,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,foetida,-1,No author
495,Crépis,sancta,sancta,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,sancta,-1,No author
671,Galium,thiebautii,np.NaN,np.NaN,np.NaN,[Ehrend.],Ehrend.,Galium,thiebautii,-1,No Mout. author
737,Hyoscyamus,albu^L.,np.NaN,np.NaN,np.NaN,[L.],L.,Hyoscyamus,albus,-1,No Mout. author
781,Knautia,integrifolia,integrifolia,np.NaN,np.NaN,np.NaN,np.NaN,Knautia,integrifolia,-1,No author
955,Myopordon,genitz,np.NaN,np.NaN,np.NaN,[Boiss.],Boiss.,Myopordon,np.NaN,-1,No Mout. author
1038,Origanum,bargyli,np.NaN,np.NaN,np.NaN,[Mouterde],Mouterde,Origanum,bargyli,-1,No Mout. author


In [118]:
vol3_no_author_combos.shape[0]

24

In [119]:
genus = "Xanthium"#"Achillea"
epithet = "strumarium"#"aleppica"
infra = "strumarium"#"aleppica"
wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet) & (wcvp_df['infraspecies'] == epithet)]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list
273654,77194547-1,Asteraceae,Xanthium,strumarium,strumarium,Xanthium strumarium subsp. strumarium,NaN,SUBSPECIES,Synonym,260893-1,Xanthium strumarium,L.,NaN,NaN,NaN,In review,NaN,NaN,NaN


In [120]:
genus = "Galium"
epithet = "canum"
wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet) & (wcvp_df['infraspecies'].isna())]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list
1112424,749676-1,Rubiaceae,Galium,canum,NaN,Galium canum,Req. ex DC.,SPECIES,Accepted,NaN,NaN,NaN,325955-2,Galium,L.,Reviewed,Prodr. 4: 602 (1830),NaN,[Req.]


## Untitled